In [2]:
# Installer les bibliothèques nécessaires
!pip install transformers datasets scikit-learn pandas




In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install sentence-transformers
!pip install pandas


In [7]:
import pandas as pd

# Charger les données train et test
train_df = pd.read_csv('/content/drive/MyDrive/train.csv')
test_df = pd.read_csv('/content/drive/MyDrive/test.csv')

# Afficher les premières lignes pour vérifier la structure
print(train_df.head())


                                            question  \
0  Je suis travailleur salarié(e). Puis-je refuse...   
1  Je suis travailleur salarié(e). Puis-je refuse...   
2  Je suis travailleur salarié(e). Puis-je refuse...   
3  Je suis travailleur salarié(e). Puis-je refuse...   
4  Je suis travailleur salarié(e). Puis-je refuse...   

                                             article  
0  Les dispositions du présent titre s'appliquent...  
1  Les travailleuses visées à l'article X.5-1, al...  
2  Lorsqu'une personne occupe des domestiques et ...  
3  L'employeur effectue l'analyse des risques vis...  
4  Les résultats de ladite analyse des risques et...  


In [9]:
from sentence_transformers import SentenceTransformer

# Charger un modèle pré-entraîné de CamemBERT
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device='cuda')  # Utilise 'cpu' si tu n'as pas de GPU

# Encoder les questions et les articles
train_embeddings_questions = model.encode(train_df['question'].tolist())
train_embeddings_articles = model.encode(train_df['article'].tolist())

test_embeddings_questions = model.encode(test_df['question'].tolist())
test_embeddings_articles = model.encode(test_df['article'].tolist())


In [10]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculer la similarité cosinus entre les embeddings des questions et des articles pour les données d'entraînement
train_similarity = cosine_similarity(train_embeddings_questions, train_embeddings_articles)

# Calculer la similarité cosinus pour les données de test
test_similarity = cosine_similarity(test_embeddings_questions, test_embeddings_articles)
from sklearn.metrics.pairwise import cosine_similarity

# Calculer la similarité cosinus entre les embeddings des questions et des articles pour les données d'entraînement
train_similarity = cosine_similarity(train_embeddings_questions, train_embeddings_articles)

# Calculer la similarité cosinus pour les données de test
test_similarity = cosine_similarity(test_embeddings_questions, test_embeddings_articles)


In [12]:
from sklearn.linear_model import LinearRegression

# Utiliser la similarité directement comme score de pertinence pour l'entraînement
train_relevance_scores = train_similarity.max(axis=1)  # Choisir la similarité la plus élevée pour chaque question

# Créer un modèle de régression
regression_model = LinearRegression()

# Entraîner le modèle de régression avec la similarité comme entrée et le score de pertinence comme étiquette
regression_model.fit(train_similarity, train_relevance_scores)


LinearRegression()

In [17]:
import numpy as np  # Import numpy and assign it the alias 'np'

# Reshape test_similarity to have the same number of features as train_similarity
# Assuming train_similarity has shape (num_train_questions, num_train_articles)
# and test_similarity has shape (num_test_questions, num_test_articles)

# Get the number of features (articles) from the training data
num_train_articles = train_similarity.shape[1]

# Create a new array with the correct shape, filled with zeros
test_similarity_reshaped = np.zeros((test_similarity.shape[0], num_train_articles)) # Now np is defined and can be used

# Copy the values from test_similarity to the new array
test_similarity_reshaped[:, :test_similarity.shape[1]] = test_similarity

# Now use the reshaped similarity matrix for prediction:
test_predictions = regression_model.predict(test_similarity_reshaped)

In [18]:
print(test_predictions)

[0.59200815 0.94070674 0.94070674 ... 0.74514349 0.81462625 0.81462625]


In [19]:
# Afficher les premières prédictions pour vérifier
print(test_predictions[:10])  # Affiche les 10 premières prédictions


[0.59200815 0.94070674 0.94070674 0.58914805 0.58914805 0.58914805
 0.93893858 0.93893858 0.93893858 0.93893858]


In [21]:
import pandas as pd

# Créer un DataFrame avec les ID de test et les prédictions
results_df = pd.DataFrame({
    'Id': test_df['Id'],  # Assurez-vous que 'id' existe dans test_df
    'relevance_score': test_predictions
})

# Afficher les premiers résultats
print(results_df.head())


   Id  relevance_score
0   0         0.592008
1   1         0.940707
2   2         0.940707
3   3         0.589148
4   4         0.589148


In [2]:
# Mettre à jour huggingface_hub et sentence_transformers
!pip install --upgrade huggingface_hub
!pip install --upgrade sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 59.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.31.0
    Uninstalling transformers-4.31.0:
      Successfully uninstalled transformers-4.31.0
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 2.2.2
    Uninstalling sentence-transformers-2.2.2:
      Successfully uninstalled sentence-transformers-2.2.2


In [9]:
!pip install sentence-transformers==2.2.2 datasets==2.11.0 transformers==4.31.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 6.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 83.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 12.2 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=3dc3cd72a450117fe965154ed4c0be6e1f4f52aa3bf7bbc90e3c073f97fc8d51
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f023953

In [7]:
!pip install --upgrade sentence-transformers datasets transformers

In [2]:
from sentence_transformers import SentenceTransformer
# Charger un modèle pré-entraîné de CamemBERT
model = SentenceTransformer('camembert-base')  # Utilise 'cpu' si tu n'as pas de GPU


In [1]:
from sentence_transformers import SentenceTransformer

# Utiliser un modèle multilingue pour des tâches de similarité de texte
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2', device='cuda')  # 'cpu' si pas de GPU


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]